<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
print(response.status_code)

200


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [4]:
 parameters = {'lat': -33.87, 'lon': 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [5]:
response = requests.get('http://api.open-notify.org/iss-pass.json', params=parameters)
print(response.json())

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1643697650, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 538, 'risetime': 1643720795}, {'duration': 659, 'risetime': 1643726518}, {'duration': 522, 'risetime': 1643732431}, {'duration': 350, 'risetime': 1643738417}, {'duration': 465, 'risetime': 1643744268}]}


Print the response header:

In [6]:
response.headers

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 01 Feb 2022 06:40:50 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}

Print the content of the response (the data that the server returned):

In [7]:
contents = response.content
print(contents)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1643697650, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 538, \n      "risetime": 1643720795\n    }, \n    {\n      "duration": 659, \n      "risetime": 1643726518\n    }, \n    {\n      "duration": 522, \n      "risetime": 1643732431\n    }, \n    {\n      "duration": 350, \n      "risetime": 1643738417\n    }, \n    {\n      "duration": 465, \n      "risetime": 1643744268\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [8]:
response.raw.read()

b''

Print just the "content-type" value from the header:

In [9]:
print(response.headers["Content-Type"]) 

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
value = contents.decode('utf-8')
print(value)

{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1643697650, 
    "latitude": -33.87, 
    "longitude": 151.21, 
    "passes": 5
  }, 
  "response": [
    {
      "duration": 538, 
      "risetime": 1643720795
    }, 
    {
      "duration": 659, 
      "risetime": 1643726518
    }, 
    {
      "duration": 522, 
      "risetime": 1643732431
    }, 
    {
      "duration": 350, 
      "risetime": 1643738417
    }, 
    {
      "duration": 465, 
      "risetime": 1643744268
    }
  ]
}



What kind of object did this give us?

In [11]:
type(value)

str

Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [12]:
res = response.json()['response']
print(res)

[{'duration': 538, 'risetime': 1643720795}, {'duration': 659, 'risetime': 1643726518}, {'duration': 522, 'risetime': 1643732431}, {'duration': 350, 'risetime': 1643738417}, {'duration': 465, 'risetime': 1643744268}]


Now extract the `risetime` strings into an array called `srisetimes`:

In [13]:
import pandas as pd
df = pd.DataFrame(res)
df

,duration,risetime
0,538,1643720795
1,659,1643726518
2,522,1643732431
3,350,1643738417
4,465,1643744268


In [14]:
srisetime = df.risetime
srisetime

0    1643720795
1    1643726518
2    1643732431
3    1643738417
4    1643744268
Name: risetime, dtype: int64

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [15]:
srisetime.dtypes

dtype('int64')

In [16]:
datetime.fromtimestamp(1643637236)

datetime.datetime(2022, 2, 1, 0, 53, 56)

In [17]:
risetime = []
for i in srisetime:
    risetime.append(datetime.fromtimestamp(i))

print(risetime)
    

[datetime.datetime(2022, 2, 2, 0, 6, 35), datetime.datetime(2022, 2, 2, 1, 41, 58), datetime.datetime(2022, 2, 2, 3, 20, 31), datetime.datetime(2022, 2, 2, 5, 0, 17), datetime.datetime(2022, 2, 2, 6, 37, 48)]


Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [18]:
for i in range(len(risetime)):
    print(risetime[i].strftime('%d/%m/%y %I:%M'))

02/02/22 12:06
02/02/22 01:41
02/02/22 03:20
02/02/22 05:00
02/02/22 06:37


Here is an endpoint that tells us who is onboard:

In [19]:
response = requests.get("http://api.open-notify.org/astros.json")
response.headers

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 01 Feb 2022 06:40:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '497', 'Connection': 'keep-alive', 'access-control-allow-origin': '*'}

Referring to the methods used above, extract the number of astronauts and their names:

In [20]:
# check content
response.content

b'{"people": [{"craft": "ISS", "name": "Mark Vande Hei"}, {"craft": "ISS", "name": "Pyotr Dubrov"}, {"craft": "ISS", "name": "Anton Shkaplerov"}, {"craft": "Shenzhou 13", "name": "Zhai Zhigang"}, {"craft": "Shenzhou 13", "name": "Wang Yaping"}, {"craft": "Shenzhou 13", "name": "Ye Guangfu"}, {"craft": "ISS", "name": "Raja Chari"}, {"craft": "ISS", "name": "Tom Marshburn"}, {"craft": "ISS", "name": "Kayla Barron"}, {"craft": "ISS", "name": "Matthias Maurer"}], "message": "success", "number": 10}'

In [21]:
people = response.json()['people']
people

[{'craft': 'ISS', 'name': 'Mark Vande Hei'},
 {'craft': 'ISS', 'name': 'Pyotr Dubrov'},
 {'craft': 'ISS', 'name': 'Anton Shkaplerov'},
 {'craft': 'Shenzhou 13', 'name': 'Zhai Zhigang'},
 {'craft': 'Shenzhou 13', 'name': 'Wang Yaping'},
 {'craft': 'Shenzhou 13', 'name': 'Ye Guangfu'},
 {'craft': 'ISS', 'name': 'Raja Chari'},
 {'craft': 'ISS', 'name': 'Tom Marshburn'},
 {'craft': 'ISS', 'name': 'Kayla Barron'},
 {'craft': 'ISS', 'name': 'Matthias Maurer'}]

In [22]:
# extract the number of astronauts and their names : 10 astronauts
df = pd.DataFrame(people)
names = df.name
print(names)

0      Mark Vande Hei
1        Pyotr Dubrov
2    Anton Shkaplerov
3        Zhai Zhigang
4         Wang Yaping
5          Ye Guangfu
6          Raja Chari
7       Tom Marshburn
8        Kayla Barron
9     Matthias Maurer
Name: name, dtype: object


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [23]:
#ANSWER:
def handleResponse(response, verbose = False):
    
    if response.status_code == 200:
        verbose = False
        status = 200 
    else :
        verbose = True
        status = response.status_code 
    return verbose, status
    
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [24]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

True

>

>

>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



